# Getting started with Gambit

In this tutorial, we'll demo the basic features of the Gambit library for game theory.

This includes creating a `Game` object and using it to set up both normal and extensive form games, starting with the Prisoner's Dilemma, one of the most famous games in game theory.

We'll then use Gambit's built-in functions to analyze the game and find its Nash equilibria.

<details><summary>The Prisoner's Dilemma</summary>

The Prisoner's Dilemma is a classic example in game theory that illustrates why two rational individuals who cannot communicate might not cooperate, even if it appears that it is in their best interest to do so. After being caught, the two prisoners are separately offered a deal:

If both stay silent (cooperate), they get light sentences.

If one betrays (defects) while the other stays silent, the betrayer goes free and the silent one gets a heavy sentence.

If both betray, they both get moderate sentences.

</details>

In [48]:
import pygambit as gbt

First, let's create the game object.

To do this, we need to know the number of players, which in Prisoner's Dilemma is 2, and the number of strategies for each player, which is in both cases is 2 (Cooperate and Defect).

In [49]:
# Create a list as long as the number of players, specifying the number of strategies for each player.
n_strategies = [2, 2]
g = gbt.Game.new_table(n_strategies, title="Prisoner's Dilemma")
type(g)

pygambit.gambit.Game

Now let's name the players and each of their possible strategies, in both cases "Cooperate" and "Defect".

In [50]:
g.players[0].label = "Tom"
g.players[0].strategies[0].label = "Cooperate"
g.players[0].strategies[1].label = "Defect"

g.players[1].label = "Jerry"
g.players[1].strategies[0].label = "Cooperate"
g.players[1].strategies[1].label = "Defect"

Now let's assign payoffs for each of the game's possible outcomes, based on the standard payoffs for the Prisoner's Dilemma:
- Both players cooperate and receive the lightest sentence: `(-1, -1)`
- Tom cooperates, but Jerry defects (betrays Tom): `(0, -3)`
- Tom defects, Jerry cooperates: `(-3, 0)`
- Both defect: `(-2, -2)`

In [51]:
# Both cooperate
g["Cooperate", "Cooperate"]["Tom"] = -1
g["Cooperate", "Cooperate"]["Jerry"] = -1

# Tom cooperates, Jerry defects
g["Cooperate", "Defect"]["Tom"] = -3
g["Cooperate", "Defect"]["Jerry"] = 0

# Tom defects, Jerry cooperates
g["Defect", "Cooperate"]["Tom"] = 0
g["Defect", "Cooperate"]["Jerry"] = -3

# Both defect
g["Defect", "Defect"]["Tom"] = -2
g["Defect", "Defect"]["Jerry"] = -2

In [52]:
# View the payout matrix
g

,Cooperate,Defect
Cooperate,"-1,-1","-3,0"
Defect,"0,-3","-2,-2"


The payout matrix structure shows what in Game Theory is described as the "strategic form" (also "normal form") representation of a game.

The matrix presents the players' strategies and their expected payoff following their played strategies.

The strategic form assumes players choose their strategies simultaneously, and the outcome depends on the combination.

## With fewer lines of code...

The most direct way to create a strategic form game is via `Game.from_arrays()`.

This function takes one n-dimensional array per player, where n is the number of players in the game.

The arrays can be any object that can be indexed like an n-times-nested Python list; so, for example, numpy arrays can be used directly.

To create a two-player symmetric game, we can simply transpose the payoff matrix for the second player before passing to `Game.from_arrays()`.

In [90]:
import numpy as np
player1_payoffs = np.array([[-1, -3], [0, -2]])
player2_payoffs = np.transpose(player1_payoffs)

g1 = gbt.Game.from_arrays(
    player1_payoffs,
    player2_payoffs,
    title="Another Prisoner's Dilemma"
)

g1

,1,2
1,"-1,-1","-3,0"
2,"0,-3","-2,-2"


Computing the Nash equilibria
-----------------------------

Let's now use Gambit to compute the Nash equilibria for our Prisoner's Dilemma game, which tells us the strategies that players can adopt to maximize their payoffs, given the assumptions of the Prisoner's Dilemma.

For a two-player normal form game, let's use `enumpure_solve` to search for a pure-strategy Nash equilibria.

In [53]:
# Returns a NashComputationResult
result = gbt.nash.enumpure_solve(g)

Let's inspect our result further to see how many equilibria were found.

For a given equilibria, we can then look at the "mixed strategy profile", which maps each strategy in a game to the corresponding probability with which that strategy is played.

Finally, we can show the expected payoffs for each player when playing the strategies as specified by an equilibrium profile.

In [54]:
# How many equilibria were found?
len(result.equilibria)

1

In [65]:
# Inspect the mixed strategy profile of the found equilibrium
msp = result.equilibria[0]
msp

[[Rational(0, 1), Rational(1, 1)], [Rational(0, 1), Rational(1, 1)]]

The equilibrium profile `[[0,1],[0,1]]` indicates that both players' strategy is to play "Cooperate" with probability 0 and "Defect" with probability 1:

In [76]:
for player in g.players:
    print(f"{player.label} plays the equilibrium strategy:")
    print(f"Probability of cooperating: {msp[player.label]['Cooperate']}")
    print(f"Probability of defecting: {msp[player.label]['Defect']}")
    print(f"Payoff: {msp.payoff(player.label)}")
    print()

Tom plays the equilibrium strategy:
Probability of cooperating: 0
Probability of defecting: 1
Payoff: -2

Jerry plays the equilibrium strategy:
Probability of cooperating: 0
Probability of defecting: 1
Payoff: -2



The equilibrium shows that both players are playing their dominant strategy, which is to defect. This is because defecting is the best response to the other player's strategy, regardless of what that strategy is.

## Extensive form games

In the Prisoner's Dilemma example above, we showed how Gambit can be used to set up a normal form game.

Gambit can also be used to set up extensive form games; the game is represented as a tree, where each node represents a decision point for a player, and the branches represent the possible actions they can take.

### Example: One-shot trust game with binary actions

[Kre90](#kre90) introduced a game commonly referred to as the **trust game**.
We will build a one-shot version of this game using Gambit's game transformation operations.

The game can be defined as follows:
- There are two players, a **Buyer** and a **Seller**.
- The Buyer moves first and has two actions, **Trust** or **Not trust**.
- If the Buyer chooses **Not trust**, then the game ends, and both players receive payoffs of `0`.
- If the Buyer chooses **Trust**, then the Seller has a choice with two actions, **Honor** or **Abuse**.
- If the Seller chooses **Honor**, both players receive payoffs of `1`;
- If the Seller chooses **Abuse**, the Buyer receives a payoff of `-1` and the Seller receives a payoff of `2`.

We create a game with an extensive representation using `Game.new_tree`:

In [91]:
g2 = gbt.Game.new_tree(
    players=["Buyer", "Seller"],
    title="One-shot trust game, after Kreps (1990)"
)

The tree of the game contains just a root node, with no children:

In [93]:
g2.root

Node(game=Game(title='One-shot trust game, after Kreps (1990)'), path=[])

In [95]:
g2.root.children

NodeChildren(parent=Node(game=Game(title='One-shot trust game, after Kreps (1990)'), path=[]))

To extend a game from an existing terminal node, use `Game.append_move`:

In [97]:
g2.append_move(g2.root, "Buyer", ["Trust", "Not trust"])
g2.root.children

NodeChildren(parent=Node(game=Game(title='One-shot trust game, after Kreps (1990)'), path=[]))

We can then also add the Seller's move in the situation after the Buyer chooses Trust:

In [99]:
g2.append_move(g2.root.children[0], "Seller", ["Honor", "Abuse"])

Now that we have the moves of the game defined, we add payoffs.

Payoffs are associated with an `Outcome`; each `Outcome` has a vector of payoffs, one for each player, and optionally an identifying text label.

First we add the outcome associated with the Seller proving themselves trustworthy:

In [101]:
g2.set_outcome(g2.root.children[0].children[0], g2.add_outcome([1, 1], label="Trustworthy"))

Next, the outcome associated with the scenario where the Buyer trusts but the Seller does not return the trust:

In [102]:
g2.set_outcome(g2.root.children[0].children[1], g2.add_outcome([-1, 2], label="Untrustworthy"))

And, finally the outcome associated with the Buyer opting out of the interaction:

In [103]:
g2.set_outcome(g2.root.children[1], g2.add_outcome([0, 0], label="Opt-out"))

Nodes without an outcome attached are assumed to have payoffs of zero for all players.

Therefore, adding the outcome to this latter terminal node is not strictly necessary in Gambit, but it is useful to be explicit for readability.

In [ ]:
# Show tree (this functionality is not yet implemented)
g2

Game(title='One-shot trust game, after Kreps (1990)')

<a id="kre90"></a> Kreps, D. (1990) “Corporate Culture and Economic Theory.” In J. Alt and K. Shepsle, eds., *Perspectives on Positive Political Economy*, Cambridge University Press.